In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format) # 항상 float 형식으로

import numpy as np 

# df plot
import cufflinks as cf 
import plotly.plotly as py 
import matplotlib.pyplot as plt 
import seaborn as sns 

# warnings 
import warnings ; warnings.filterwarnings("ignore")

# style
plt.style.use("ggplot")
cf.go_offline()

In [2]:
# 한글 폰트 출력 
import matplotlib
from matplotlib import font_manager, rc 
import platform 

font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc("font",family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

In [3]:
# 데이터 로드 
path = "C:/Users/sk_jo/Desktop/Project/titanic/"

def load_dataset(): 
    sales = pd.read_excel(path+"sales.xlsx",header =1)
    view_rate = pd.read_excel(path+"view_rate.xlsx",header =1, index_col= 0)
    return sales, view_rate

sales = load_dataset()[0]
view_rate = load_dataset()[1]

# 데이터 전처리 

In [4]:
## TODO 
## 1.무형 상품은 추정 제외 
## 2.결측된 노출분 채우기 (직전의 값 집어넣기) 
## 3. 판매단가 > 취급액이면 취급액 0 
##

def custom(price, total):
    # 3전처리 
    if price > total: return 0
    else: return total

def pre_processing(df): 
    # 1. 무형 상품 추정 제외 
    df = df.drop(df[df["상품군"]=="무형"].index,axis = 0)
    # 2. 결측된 노출분 채우기 
    for i,row in df.iterrows():
        val = row["노출(분)"]
        if i == 0:
            continue 
        if np.isnan(val):
            if df.loc[i,"방송일시"] == df.loc[i-1,"방송일시"]:
                df.loc[i,"노출(분)"] = df.loc[i-1,"노출(분)"]
            else :
                continue 
                
    # 3. 판매단가 > 취급액이면 취급액 0 
    df["취급액"] = df.apply(lambda x : custom(x["판매단가"], x["취급액"]),axis=1)
    print(sum((sales["판매단가"] > sales["취급액"]) & (sales['취급액'] != 0)))
    # 4. month, day, hour, minute, weekday 열 추가 
    df["방송일시"] = pd.to_datetime(df["방송일시"])
    df['month'] = pd.DatetimeIndex(df['방송일시']).month
    df['day'] = pd.DatetimeIndex(df['방송일시']).day
    df['hour'] = pd.DatetimeIndex(df['방송일시']).hour
    df['minute'] = pd.DatetimeIndex(df['방송일시']).minute
    df['weekday'] = pd.DatetimeIndex(df['방송일시']).weekday
    
    # reset_index
    df = df.reset_index(drop=True)
    
    return df 

sales = pre_processing(sales)

1990


In [24]:
weather = pd.read_csv("./weathers.csv")
weather.head()

,방송일시,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),미세먼지 주의보 및 경보,평균기온(℃),최고기온(℃),최저기온(℃),season
0,2019-01-01,39.36,26.12,0.01,0.03,0.69,0.00,0,-5.00,-0.60,-8.20,겨울
1,2019-01-02,33.08,21.36,0.01,0.04,0.68,0.00,0,-4.90,0.20,-8.80,겨울
2,2019-01-03,38.80,23.36,0.01,0.04,0.74,0.01,0,-3.50,3.20,-8.40,겨울
3,2019-01-04,59.44,40.36,0.00,0.06,1.06,0.01,0,-1.10,4.10,-6.20,겨울
4,2019-01-05,65.72,42.52,0.02,0.03,0.64,0.01,1,-2.80,1.10,-5.50,겨울


In [25]:
sales["판매량"] = sales["취급액"]/sales["판매단가"]
sales = sales[["판매량","취급액","weekday","방송일시"]]
weather = weather[["방송일시","season"]]

In [29]:
weather = weather.rename(columns = {"방송일시":"방송날짜"})

In [32]:
import datetime 

sales["방송날짜"] = sales["방송일시"].apply(lambda x : datetime.datetime.strftime(x,"%Y-%m-%d"))

In [33]:
sales.head()

,판매량,취급액,weekday,방송일시,방송날짜
0,52.61,2099000.00,1,2019-01-01 06:00:00,2019-01-01
1,109.55,4371000.00,1,2019-01-01 06:00:00,2019-01-01
2,81.75,3262000.00,1,2019-01-01 06:20:00,2019-01-01
3,174.31,6955000.00,1,2019-01-01 06:20:00,2019-01-01
4,167.22,6672000.00,1,2019-01-01 06:40:00,2019-01-01


In [44]:
new_df = pd.merge(sales,weather,how = "inner", on = "방송날짜")

In [45]:
new_df["방송시간"] = new_df["방송일시"].apply(lambda x : datetime.datetime.strftime(x,"%H:%M"))
new_df.head()

,판매량,취급액,weekday,방송일시,방송날짜,season,방송시간
0,52.61,2099000.00,1,2019-01-01 06:00:00,2019-01-01,겨울,06:00
1,109.55,4371000.00,1,2019-01-01 06:00:00,2019-01-01,겨울,06:00
2,81.75,3262000.00,1,2019-01-01 06:20:00,2019-01-01,겨울,06:20
3,174.31,6955000.00,1,2019-01-01 06:20:00,2019-01-01,겨울,06:20
4,167.22,6672000.00,1,2019-01-01 06:40:00,2019-01-01,겨울,06:40


season별 주중 주말 데이터의 시간대별 판매량 및 취급액 차이

In [47]:
fall = new_df[new_df["season"]=="가을"]

In [53]:
fall[fall["weekday"].isin([5,6])].groupby(["방송시간"]).sum()["취급액"].iplot()

In [60]:
def visualize(df,season, y = "판매량"):
    # new_df 
    # season이름 
    df = df[df["season"]==season]
    weekday = df[df["weekday"].isin([0,1,2,3,4])]
    weekend = df[df["weekday"].isin([5,6])]
    
    weekday.groupby(["방송시간"]).sum()[y].iplot(kind = "bar",title= "{}의 주중 시간별 {} 총합".format(season,y))
    weekend.groupby(["방송시간"]).sum()[y].iplot(kind = "bar",colors = "blue",title= "{}의 주말 시간별 {} 총합".format(season,y))
    
        
        

visualize(new_df,"봄",y="취급액")